# **CRM Analytics - Data Cleaning**

**Impoting libraries and modules**

In [1]:
# Main libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

# Dealing with warnings
import warnings
warnings.filterwarnings('ignore')

# Setting up DataFrame settings
pd.set_option('display.max_columns', 50)

## Data Cleaning

**Importing CSV files into Jupyter Notebook**

In [6]:
cases = pd.read_csv('data/cases.csv', date_parser = ['date_ref']) # Cases
creds = pd.read_csv('data/creds.csv', date_parser = ['cred_date']) # Credenciamentos (registration)

### Cleaning `cases` dataset

In [7]:
cases.head(5)

,Unnamed: 0,accountid,date_ref,channelid,waitingtime,missed,pesquisa_de_satisfa_o__c,assunto,Id
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,0013j00002z0CeEAAU,2020-07-31,2.0,15.0,False,NaN,Aplicativo:Dúvidas funcionalidades App:Primeir...,0013j00002z0CeEAAU


**The `cases` dataset is not in the best format for our analysis.**

To clean the data, the following steps will be necessary:

1. Drop column `Unnamed: 0`;
2. Rename columns to facilitate the handling of the dataset;
3. Remove entries where all values are null;
4. Parse the `date_ref` column for into datetime format;
5. Organize entries by `data_ref`, in ascending order;
6. Check if the `accountid` (PK) and` Id` columns are redundant and, if so, eliminate `Id`;
7. Study values in `channelid` and` wairingtime`, and, if necessary, transform them into `int`;
8. Check for duplicate entries and, if so, remove redundancies;
9. Study the `subject` column and understand the best way to store information in a structured way and apply it;

#### Renaming columns

In [9]:
cases.columns = ['unnamed',
                 'account_id', 
                 'date_ref',
                 'channel_id',
                 'waiting_time',
                 'missed', 
                 'pesq_satisfacao', 
                 'assunto', 
                 'id']

cases.head(3)

,unnamed,account_id,date_ref,channel_id,waiting_time,missed,pesq_satisfacao,assunto,id
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Dropping`unnamed`

In [10]:
# Drop "Unnamed: 0"
cases.duplicated('unnamed').sum() # Checking duplicates in dataset index
cases = cases.drop('unnamed', axis = 1)

#### Removing null values

Removing entries where _all_ values are nulls:

In [11]:
cases['account_id'].isna().sum() # PK Cases missing

49500

In [12]:
# Removing null values
ccases = cases.dropna(how = 'all', axis = 0) # ccases = cleaned cases
ccases.shape

# Reporting changes
print("Qtd entradas dataset original:        ", len(cases), 
      "\nQtd entradas dataset sem NaNs:         ", len(ccases), 
      "\nQtd entradas descartadas após dropna:  ", (len(cases)-len(ccases)))

Qtd entradas dataset original:         126989 
Qtd entradas dataset sem NaNs:          77489 
Qtd entradas descartadas após dropna:   49500


In [13]:
# Checking remaining nulls
ccases.isna().sum()

account_id             0
date_ref               0
channel_id             0
waiting_time           0
missed                 0
pesq_satisfacao    65904
assunto                0
id                     0
dtype: int64

#### Investigating `account_id`

**Question: does `account_id` meet the requirements to be the dataset's Primary Key?**

According to the dictionary, `account_id` should be the table's primary key (PK), to meet this requirement, it must be unique in each of the entries.

However, it seems reasonable to think that in the timespan of the dataset, the same customer (each with a unique account id, may have made several calls in the Customer Relationship center (CR).

Let's check:

In [14]:
# Checking 'account_id' duplicates
cases.duplicated('account_id').value_counts()

True     95559
False    31430
dtype: int64

In [15]:
cases[cases.duplicated('account_id')]

,account_id,date_ref,channel_id,waiting_time,missed,pesq_satisfacao,assunto,id
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,0013j00002z0CeEAAU,2020-07-31,2.0,15.0,False,NaN,Produto:S920:Ativação,0013j00002z0CeEAAU
6,0013j00002z0CeEAAU,2020-09-23,2.0,1.0,False,NaN,Produto:Cartão pré-pago:Dúvidas,0013j00002z0CeEAAU
...,...,...,...,...,...,...,...,...
126984,0013j00002yi49FAAQ,2020-08-26,2.0,9.0,False,NaN,Aplicativo:Dúvidas funcionalidades App:Prazos ...,0013j00002yi49FAAQ
126985,0013j00002yi49FAAQ,2020-09-09,2.0,4.0,False,NaN,Produto:D195:Ativação,0013j00002yi49FAAQ
126986,0013j00002yi49FAAQ,2020-09-21,2.0,3.0,False,NaN,Cadastro:Recredenciamento:Dúvidas,0013j00002yi49FAAQ
126987,0013j00002yi49FAAQ,2020-09-21,2.0,3.0,False,NaN,Produto:Cartão pré-pago:Dúvidas,0013j00002yi49FAAQ


**31,430 _False_ values** indicates that, as previously thought, several accounts have called the CR more than once. Therefore, this cannot be the table's primary key.

As there is no other one that can occupy this position, we will keep the index of the dataset as PK, where each line represents a call to the company.

#### Ordering dataset by  `date_ref`

In [16]:
# Ordering dataset
ccases = ccases.sort_values('date_ref')

#### Checking redundancy
Checking if the information in the `account_id` and `id` columns is the same.

In [17]:
# Checking if there are different values in account_id and id
(ccases['account_id'] != ccases['id']).sum()

0

In [18]:
# Dropping reduntand 'id' column
ccases = ccases.drop('id', axis = 1)
# ccases.head(3)

#### Reviewing `channel_id`
Check values in `channel_id` and parse them into ` int`, if applicable.

In [20]:
ccases['channel_id'].value_counts() # Only one value found; According to the dictionary, this channel represents
                                    # calls received by phone. The column can be droped because the analysis is 
                                    # now specific to that channel.

2.0    77489
Name: channel_id, dtype: int64

In [21]:
# Dropping "channel_id"
ccases.drop('channel_id', axis = 1, inplace = True)
# ccases.head(3)

#### Checking duplicated entries

In [22]:
ccases.duplicated().sum() # There are no duplicates

0

In [23]:
ccases = ccases.reset_index(drop = True)
ccases.head(5)

,account_id,date_ref,waiting_time,missed,pesq_satisfacao,assunto
0,0011L00002ZbpnlQAB,2020-02-23,19.0,False,Enviado,Produto:mPOS:Dúvidas mpos
1,0011L00002dbBg5QAE,2020-02-25,15.0,False,Enviado,Aplicativo:Problema:
2,0011L00002WdgbcQAB,2020-02-26,15.0,False,Enviado,Aplicativo:Dúvidas funcionalidades App:Redefin...
3,0011L00002WdJgjQAF,2020-02-26,13.0,False,Enviado,Produto:mPOS:Problema POS - revertido
4,0011L00002We7cjQAB,2020-02-26,72.0,False,Enviado,Aplicativo::


In [24]:
# ccases.tail(5)

#### Parsing `date_ref`

In [25]:
# Parsing "date_ref" to datetime object
ccases['date_ref'] = pd.to_datetime(ccases['date_ref'])
# ccases.info()

#### Feature engineering of temporal data
Transforming `waiting_time` column (measured in seconds) to` int` and `timedelta` (for date and time operations)

In [26]:
ccases['waiting_time'] = ccases['waiting_time'].astype(int)

In [27]:
ccases['waiting_time'] = ccases['waiting_time'].astype(int)
ccases['waiting_timedelta'] = pd.to_timedelta(ccases['waiting_time'], unit = 'S')

In [30]:
def explode_date(df, date_column, language_ptbr = True):
    '''Generates year, month, day and weekday from a single date column.
        
    :::::::
    
    df:            DataFrame containing the date column to be exploded.
    date_column:   Date column (Series) to be used as base for the transformation.
    language_ptbr: If True, parse weekday names to Brazilian Portuguese. 
                   False, weekdays will be parsed to English.
    :::::::

    '''
    df[date_column] = pd.to_datetime(df[date_column])
    df['year'] = df[date_column].dt.year 
    df['month'] = df[date_column].dt.month
    df['day'] = df[date_column].dt.day
    df['weekday'] = df[date_column].dt.weekday
    
    if language_ptbr:
        df['weekday_str'] = df['weekday'].map({
                                    0: 'Seg',
                                    1: 'Ter',
                                    2: 'Qua',
                                    3: 'Qui',
                                    4: 'Sex',
                                    5: 'Sab',
                                    6: 'Dom'
                            })
    else:
        df['weekday_str'] = df[date_column].dt.day_name().str[0:3]

    return df

In [31]:
ccases = explode_date(ccases, 'date_ref', language_ptbr = True)
ccases.head(3)

,account_id,date_ref,waiting_time,missed,pesq_satisfacao,assunto,waiting_timedelta,year,month,day,weekday,weekday_str
0,0011L00002ZbpnlQAB,2020-02-23,19,False,Enviado,Produto:mPOS:Dúvidas mpos,0 days 00:00:19,2020,2,23,6,Dom
1,0011L00002dbBg5QAE,2020-02-25,15,False,Enviado,Aplicativo:Problema:,0 days 00:00:15,2020,2,25,1,Ter
2,0011L00002WdgbcQAB,2020-02-26,15,False,Enviado,Aplicativo:Dúvidas funcionalidades App:Redefin...,0 days 00:00:15,2020,2,26,2,Qua


#### Structuring data in `assunto` ("subject", in English)

In [32]:
# Separating chained topics and saving them in a list
ccases['assunto'] = ccases['assunto'].str.split(':')

In [34]:
# Creating specific columns for each topic (calling them nodes)
ccases[['node_1','node_2', 'node_3']] = pd.DataFrame(ccases['assunto'].tolist(), index = ccases.index)

# Filling null valiues with NaNs
ccases[['node_1', 'node_2', 'node_3']] = ccases[['node_1', 'node_2', 'node_3']].replace('', np.nan)
ccases.head(3)

,account_id,date_ref,waiting_time,missed,pesq_satisfacao,assunto,waiting_timedelta,year,month,day,weekday,weekday_str,node_1,node_2,node_3
0,0011L00002ZbpnlQAB,2020-02-23,19,False,Enviado,"[Produto, mPOS, Dúvidas mpos]",0 days 00:00:19,2020,2,23,6,Dom,Produto,mPOS,Dúvidas mpos
1,0011L00002dbBg5QAE,2020-02-25,15,False,Enviado,"[Aplicativo, Problema, ]",0 days 00:00:15,2020,2,25,1,Ter,Aplicativo,Problema,NaN
2,0011L00002WdgbcQAB,2020-02-26,15,False,Enviado,"[Aplicativo, Dúvidas funcionalidades App, Rede...",0 days 00:00:15,2020,2,26,2,Qua,Aplicativo,Dúvidas funcionalidades App,Redefinição de senha


### Cleaning `creds` dataset

In [35]:
creds.head(3)

,Unnamed: 0,cred_date,shipping_address_city,shipping_address_state,max_machine,accountid
0,0,2020-04-18,Feira de Santana,BA,T1,NaN
1,1,2020-10-16,Bacuri,MA,T1,NaN
2,2,2020-09-01,Bernardo Sayão,TO,T1,NaN


#### Renaming Columns

In [36]:
creds.columns = ['unnamed', 'cred_date', 'ship_city', 'ship_state', 'max_machine', 'account_id']
creds.head(3)

,unnamed,cred_date,ship_city,ship_state,max_machine,account_id
0,0,2020-04-18,Feira de Santana,BA,T1,NaN
1,1,2020-10-16,Bacuri,MA,T1,NaN
2,2,2020-09-01,Bernardo Sayão,TO,T1,NaN


#### Dropping `unnamed`

In [37]:
ccreds = creds.drop('unnamed', axis = 1)
ccreds.head(3)

,cred_date,ship_city,ship_state,max_machine,account_id
0,2020-04-18,Feira de Santana,BA,T1,NaN
1,2020-10-16,Bacuri,MA,T1,NaN
2,2020-09-01,Bernardo Sayão,TO,T1,NaN


#### Ordering dataset by `cred_date`

In [38]:
ccreds = ccreds.sort_values(by = 'cred_date').reset_index(drop = True)

#### Parsing `cred_date`

In [39]:
explode_date(ccreds, 'cred_date', language_ptbr = True)
ccreds.head(3)

,cred_date,ship_city,ship_state,max_machine,account_id,year,month,day,weekday,weekday_str
0,2019-07-11,São Paulo,SP,NONE,NaN,2019,7,11,3,Qui
1,2019-07-23,Mogi das Cruzes,SP,NONE,NaN,2019,7,23,1,Ter
2,2019-07-23,São Luís,MA,NONE,NaN,2019,7,23,1,Ter


#### Checking duplicates

In [40]:
print(f'{ccreds.duplicated().sum()} entries must be removed.') # There are duplicated entries that need to be removed

60832 entries must be removed.


In [41]:
# Removing dupliated entries
ccreds = ccreds.drop_duplicates()
creds.head(3)

,unnamed,cred_date,ship_city,ship_state,max_machine,account_id
0,0,2020-04-18,Feira de Santana,BA,T1,NaN
1,1,2020-10-16,Bacuri,MA,T1,NaN
2,2,2020-09-01,Bernardo Sayão,TO,T1,NaN


**Some considerations**

It is an interesting situation. According to the data dictionary, this is a dataset of registered customers, counting even with a register date. As we have seen, many values were duplicated. It is possible, therefore, that there are more duplicate lines, but this information, without knowing the value `account_id`, is merely speculation. And speculation and _data-driven_ analysis are the exact opposite of each other...

In principle, I considered that these entries could represent a first contact with the relationship center, a _lead_. But most lines have an associated payment machine. If there is an associated machine, then, in my limited understanding at the moment, there would be an `account_id`.

In an ideal situation, I would like to talk to the owner of this dataset in order to understand it better and find out if it is an import error that erases some values of `account_id` or an error of some other type.

For now, I will save this data in another dataset, specific with entries without `account_id` and drop those lines from the main dataset` ccred`.

#### Saving entries with no `account_id` into a new dataset, `ccreds_naid`

In [42]:
ccreds_naid = ccreds[ccreds['account_id'].isna()]
ccreds_naid.head()

,cred_date,ship_city,ship_state,max_machine,account_id,year,month,day,weekday,weekday_str
0,2019-07-11,São Paulo,SP,NONE,NaN,2019,7,11,3,Qui
1,2019-07-23,Mogi das Cruzes,SP,NONE,NaN,2019,7,23,1,Ter
2,2019-07-23,São Luís,MA,NONE,NaN,2019,7,23,1,Ter
3,2019-07-23,Recife,PE,T1,NaN,2019,7,23,1,Ter
4,2019-07-23,Teresina,PI,NONE,NaN,2019,7,23,1,Ter


In [43]:
ccreds_naid.shape[0] # Total de entradas sem account_id

34728

**Checking entries with no associated payment machine (`max_machine` = NONE)**

In [44]:
ccreds_naid[ccreds_naid['max_machine'] == 'NONE'].shape[0] # Of 34,728 entries with no register of account_id, 
                                                           # only 752 do not have a payment machine associated with it

752

#### Removing Nulls

In [45]:
ccreds = ccreds.dropna(subset = ['account_id'])
ccreds.head(3)

,cred_date,ship_city,ship_state,max_machine,account_id,year,month,day,weekday,weekday_str
10,2019-07-24,Curitiba,PR,T1,0011L00002WdC77QAF,2019,7,24,2,Qua
12,2019-07-24,Recife,PE,T1,0011L00002WdB4gQAF,2019,7,24,2,Qua
17,2019-07-24,Betim,MG,T1,0011L00002Wd777QAB,2019,7,24,2,Qua


In [46]:
# Checking if all 'account_id' remaning entries have unique values
ccreds.shape[0] == ccreds['account_id'].nunique()

True

---

### Exporting cleaned datasets

In [47]:
# ccases.info()

In [48]:
ccases.to_csv('data/cases_cleaned.csv', index = False)
ccreds.to_csv('data/creds_cleaned.csv', index = False)
ccreds_naid.to_csv('data/creds_noids.csv', index = False)

**The next step is the analysis of the data itself.**

For better organization, it will be carried out in a new Notebook.

# TO INCLUDE IN THE ANALYSIS

In [49]:
# # Counting topics per node
# print('Qtd. tópicos no node_1:  ', ccases['node_1'].nunique(dropna = False))
# print('Qtd. tópicos no node_2:  ', ccases['node_2'].nunique(dropna = False))
# print('Qtd. tópicos no node_3: ', ccases['node_3'].nunique(dropna = False), '\n')

# print('Qtd. valores faltantes no node_1:   ', ccases['node_1'].isna().sum(), '<-- What are those?')
# print('Qtd. valores faltantes no node_2:  ', ccases['node_2'].isna().sum())
# print('Qtd. valores faltantes no node_3: ', ccases['node_3'].isna().sum())

# # Transforming topics into sets to assess whether they are subsets of each other
# topics_node_1 = set(ccases['node_1'].value_counts(dropna = False).index.tolist())
# topics_node_2 = set(ccases['node_2'].value_counts(dropna = False).index.tolist())
# topics_node_3 = set(ccases['node_3'].value_counts(dropna = False).index.tolist())

In [58]:
# # Checking if sets are subsets of each other
# topics_node_2.issubset(topics_node_1), \
# topics_node_3.issubset(topics_node_2), \
# topics_node_3.issubset(topics_node_1)   

In [59]:
# ccases[ccases['node_1'].isna()].head(10)